In [ ]:
sc.addPyFile("../Graph/graphframes-0.5.0-spark2.1-s_2.11.jar")

In [2]:
from graphframes import *
from pyspark.sql.functions import *

In [3]:
# Vertics DataFrame
v = spark.createDataFrame([
  ("a", "Alice", 34),
  ("b", "Bob", 36),
  ("c", "Charlie", 37),
  ("d", "David", 29),
  ("e", "Esther", 32),
  ("f", "Fanny", 38),
  ("g", "Gabby", 60)
], ["id", "name", "age"])

# Edges DataFrame
e = spark.createDataFrame([
  ("a", "b", "friend"),
  ("b", "c", "follow"),
  ("c", "b", "follow"),
  ("f", "c", "follow"),
  ("e", "f", "follow"),
  ("e", "d", "friend"),
  ("d", "a", "friend"),
  ("a", "e", "friend"),
  ("g", "e", "follow")
], ["src", "dst", "relationship"])

# Create a GraphFrame
g = GraphFrame(v, e)

g.vertices.show()
g.edges.show()

+---+-------+---+
| id|   name|age|
+---+-------+---+
|  a|  Alice| 34|
|  b|    Bob| 36|
|  c|Charlie| 37|
|  d|  David| 29|
|  e| Esther| 32|
|  f|  Fanny| 38|
|  g|  Gabby| 60|
+---+-------+---+

+---+---+------------+
|src|dst|relationship|
+---+---+------------+
|  a|  b|      friend|
|  b|  c|      follow|
|  c|  b|      follow|
|  f|  c|      follow|
|  e|  f|      follow|
|  e|  d|      friend|
|  d|  a|      friend|
|  a|  e|      friend|
|  g|  e|      follow|
+---+---+------------+



In [7]:
# Q2
g.find("(a)-[]->(b);(b)-[]->(c)")\
    .filter("a.name='Alice'").select("c.name").show()

+-------+
|   name|
+-------+
|  Fanny|
|  David|
|Charlie|
+-------+



In [8]:
# Q3
g.find("(a)-[]->(b);(b)-[]->(c);!(c)-[]->(a)")\
    .filter("a.name='Alice'").select("c.name").show()

+-------+
|   name|
+-------+
|  Fanny|
|Charlie|
+-------+



In [10]:
# Q4
g.find("(a)-[e1]->(b)")\
    .filter("b.name = 'Charlie' AND e1.relationship='follow'")\
    .select('a.name').show()

+-----+
| name|
+-----+
|Fanny|
|  Bob|
+-----+



In [13]:
# Q5
g.find("(a)-[e]->(b)")\
    .filter("e.relationship='follow'")\
    .groupBy('b').count().filter("count>=2").show()

+--------------+-----+
|             b|count|
+--------------+-----+
|[c,Charlie,37]|    2|
+--------------+-----+



In [15]:
from graphframes import *

# Vertices DataFrame
v = spark.createDataFrame([
 ("a", "Alice", 34),
 ("b", "Bob", 36),
 ("c", "Charlie", 37),
 ("d", "David", 29),
 ("e", "Esther", 32),
 ("f", "Fanny", 38),
 ("g", "Gabby", 60)
], ["id", "name", "age"])

# Edges DataFrame
e = spark.createDataFrame([
 ("a", "b", "follow"),
 ("c", "a", "friend"),
 ("b", "c", "follow"),
 ("d", "a", "follow"),
 ("f", "c", "follow"),
 ("f", "d", "follow"),
 ("f", "b", "follow"),
 ("c", "d", "follow"),
 ("g", "a", "friend"),
 ("g", "d", "friend"),
 ("g", "c", "friend"),
 ("e", "a", "follow"),
 ("e", "d", "follow")
], ["src", "dst", "relationship"])
# Create a GraphFrame
g = GraphFrame(v, e)

In [21]:
# Correct way:
zombies= g.vertices.join(g.inDegrees, 'id', 'left_outer').filter('inDegree is null')
follow_count=g.find("(a)-[e]->(b)")\
    .filter("e.relationship='follow'")\
    .groupBy("a").count().select("a.name","count")
follow_count.show()
zombies.show()

+-------+-----+
|   name|count|
+-------+-----+
| Esther|    2|
|    Bob|    1|
|  Fanny|    3|
|  Alice|    1|
|Charlie|    1|
|  David|    1|
+-------+-----+

+---+------+---+--------+
| id|  name|age|inDegree|
+---+------+---+--------+
|  g| Gabby| 60|    null|
|  f| Fanny| 38|    null|
|  e|Esther| 32|    null|
+---+------+---+--------+



In [24]:
answers=zombies.join(follow_count,"name")\
    .orderBy("count",ascending=False)
answers.show()

+------+---+---+--------+-----+
|  name| id|age|inDegree|count|
+------+---+---+--------+-----+
| Fanny|  f| 38|    null|    3|
|Esther|  e| 32|    null|    2|
+------+---+---+--------+-----+



In [28]:
answer=answers.select('name').rdd.map(lambda row: row.name).take(1)
print answer

[u'Fanny']
